In [1]:
from counterfactuals.explainers import Cadex, Fimap
from counterfactuals.visualization import show
from data import GermanData, AdultData
from tensorflow import keras
import numpy as np

2022-01-19 20:29:27.290888: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-01-19 20:29:27.290915: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
german_data = GermanData('data/datasets/input_german.csv', 'data/datasets/labels_german.csv')
model = keras.models.load_model('models/model_german')
test_X = german_data.X_test.iloc[6]
cadex = Cadex(model)
cf = cadex.generate(test_X)

2022-01-19 20:29:28.883584: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-01-19 20:29:28.883605: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-19 20:29:28.883623: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (localhost.localdomain): /proc/driver/nvidia/version does not exist
2022-01-19 20:29:28.883822: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
german_data.X_train.head()

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
965.0,0.734682,-0.558363,0.900274,-1.688832,-0.830331,-0.718745,-0.422141,1.644294,-0.621677,-0.246998,...,-0.494134,0.818625,-0.818625,-0.197386,0.197386,1.362770,-0.45257,-0.456573,-0.575426,-0.249834
742.0,-0.006485,-0.049081,0.900274,0.140641,0.495018,-0.718745,-0.422141,-0.608164,-0.621677,-0.246998,...,-0.494134,-1.221561,1.221561,-0.197386,0.197386,-0.733799,-0.45257,-0.456573,1.737844,-0.249834
920.0,-0.253541,-0.522414,0.900274,-0.774096,-0.653618,1.045047,-0.422141,-0.608164,-0.621677,-0.246998,...,-0.494134,0.818625,-0.818625,-0.197386,0.197386,1.362770,-0.45257,-0.456573,-0.575426,-0.249834
583.0,1.228794,-0.322578,0.900274,-1.688832,-0.211835,-0.718745,-0.422141,1.644294,-0.621677,-0.246998,...,2.023744,0.818625,-0.818625,-0.197386,0.197386,-0.733799,-0.45257,2.190229,-0.575426,-0.249834
896.0,-0.006485,-0.244335,0.900274,1.055377,-0.653618,-0.718745,-0.422141,-0.608164,1.608553,-0.246998,...,-0.494134,-1.221561,1.221561,-0.197386,0.197386,-0.733799,-0.45257,2.190229,-0.575426,-0.249834


In [4]:
df = show(test_X, cf.iloc[0])
df

,X,X',change
duration,-0.253541,1.470143,1.723684
account_status_< 0 DM,-0.621677,1.102007,1.723684
account_status_no checking account,1.240839,-0.482844,-1.723684
debtors_guarantor,-0.244137,-1.967820,-1.723684
foreign_yes,0.197386,1.921069,1.723684


In [5]:
test_X = german_data.input.iloc[6]
cadex = Cadex(model, transform=german_data.scale, inverse_transform=german_data.unscale)
cf = cadex.generate(test_X)
if cf is not None:
    show(test_X, cf.iloc[0])
else:
    print("cf not generated")

In [6]:
test_X = german_data.X_test.iloc[10]
cadex = Cadex(model, n_changed=8)
cf = cadex.generate(test_X)
show(test_X, cf.iloc[0])

,X,X',change
duration,-0.253541,0.227586,0.481127
age,0.760088,0.278961,-0.481127
account_status_< 0 DM,-0.621677,-0.140550,0.481127
account_status_no checking account,-0.805906,-1.287033,-0.481127
savings_unknown/ no savings account,-0.472484,-0.953611,-0.481126
debtors_guarantor,-0.244137,-0.725264,-0.481127
housing_own,0.658553,0.177426,-0.481126
foreign_yes,0.197386,0.678513,0.481127


In [7]:
model = keras.models.load_model('models/model_german')
model_predictions = model.predict(german_data.X_train)
model_predictions = np.argmax(model_predictions, axis=1)
fimap = Fimap()
fimap.fit(german_data.X_train, model_predictions)

test_X = german_data.X_test.iloc[0]
cf = fimap.generate(test_X)


Training s

Training g


In [8]:
df = show(test_X, cf.iloc[0])
df

,X,X',change
duration,2.217017,2.224786,0.007769
credit,1.959146,1.961757,0.002611
installment_percent,-0.895784,-0.898583,-0.002800
residence_duration,-1.688832,-1.678291,0.010540
age,-0.035122,-0.034600,0.000522
...,...,...,...
employment_1..4 years,-0.733799,-0.726584,0.007215
employment_4..7 years,2.209605,2.214446,0.004841
employment_< 1 year,-0.456573,-0.449223,0.007351
employment_>= 7 years,-0.575426,-0.580039,-0.004613


In [9]:
from counterfactuals.explainers import ECE

ece = ECE(5, columns=list(german_data.X_train.columns), bces=[cadex, fimap], dist=2, h=8, lambda_=0.2, n_jobs=1)

cfs = ece.generate(german_data.X_test.iloc[0])


/tmp/ipykernel_31564/3035201521.py:5: UserWarning: k parameter > number of aggregated counterfactuals. Changing k from 5 to 2
  cfs = ece.generate(german_data.X_test.iloc[0])
/tmp/ipykernel_31564/3035201521.py:5: UserWarning: knn's h parameter >= number of aggregated counterfactuals. Changing h from  8 to 1
  cfs = ece.generate(german_data.X_test.iloc[0])


In [10]:
cfs

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
0,2.224786,1.961757,-0.898583,-1.678291,-0.0346,1.033385,-0.418187,-0.612316,-0.61657,-0.241543,...,-0.491933,-1.215873,1.217144,-0.195348,0.194342,-0.726584,2.214446,-0.449223,-0.580039,-0.236962


In [11]:
for i in range(cfs.shape[0]):
    show(german_data.X_test.iloc[0], cfs.iloc[i, :])

In [12]:
ece.get_aggregated_cfs()

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
0,2.224786,1.961757,-0.898583,-1.678291,-0.034600,1.033385,-0.418187,-0.612316,-0.616570,-0.241543,...,-0.491933,-1.215873,1.217144,-0.195348,0.194342,-0.726584,2.214446,-0.449223,-0.580039,-0.236962
1,3.135470,1.959146,-0.895784,-1.688832,-0.953574,1.045047,-0.422141,-0.608164,0.296776,-0.246998,...,-0.494134,-1.221561,1.221561,-0.197386,1.115838,-0.733799,2.209605,-0.456573,-0.575426,-0.249834


In [13]:
german_data.X_train.iloc[0]

duration                 0.734682
credit                  -0.558363
installment_percent      0.900274
residence_duration      -1.688832
age                     -0.830331
                           ...   
employment_1..4 years    1.362770
employment_4..7 years   -0.452570
employment_< 1 year     -0.456573
employment_>= 7 years   -0.575426
employment_unemployed   -0.249834
Name: 965.0, Length: 61, dtype: float64